## Libraries

In [1]:
import sys 
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\BARTScore")
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries") 

In [2]:
from custom_score.score import BERTScoreStaticSampleTest, BERTScoreDynamicSampleTest, BARTScoreDynamicSampleTest
from custom_score.utils import serialized_to_model
from bart_score import BARTScorer
import torch
import tensorflow_datasets as tfds
from datetime import datetime
import pandas as pd
import numpy as np

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Checkup and linkage

In [3]:
torch.cuda.is_available()

True

## Datasets

### Billsum

In [3]:
billsum = tfds.load('huggingface:billsum')
billsum_test = tfds.as_dataframe(billsum["test"])
billsum_test = billsum_test.loc[:, ["text", "summary"]]
billsum_test.text = billsum_test.text.str.decode("utf-8")
billsum_test.summary = billsum_test.summary.str.decode("utf-8")
billsum_test.head(5)

,text,summary
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,Local Innovation and Coastal Protection Act of...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Gun Show Background Check Act of 2008 - Amends...
2,SECTION 1. SHORT TITLE.\n\n This Act may be...,Recycled Roads Act of 2003 - Directs the Secre...
3,SECTION 1. SHORT TITLE.\n\n This Act may be...,Prosthetic and Custom Orthotic Parity Act of 2...
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,"Investing in Neighborhood-focused, Vital, Evid..."


### Gigaword

In [28]:
gigaword_builder = tfds.builder("Gigaword")
gigaword_builder.download_and_prepare()
gigaword = gigaword_builder.as_dataset()

In [29]:
gigaword = tfds.as_dataframe(gigaword["test"])
gigaword = gigaword.rename(columns={"document":"text", "summary":"summary"})
gigaword['summary'] = gigaword['summary'].str.decode("utf-8").str.replace("\xe2\x80\x93", "").str[2:].str.strip()
gigaword['text'] = gigaword['text'].str.decode("utf-8").str.strip()
gigaword.head(5)

,text,summary
0,the german government and red cross have decid...,"rmany gives ###,### dollars in aid for iran qu..."
1,teenager michaela krajicek claimed her first g...,ajicek defeats safina in dutch final UNK picture
2,after spending ## years in prison for a killin...,"###,### for ## years wrongly held in prison"
3,the first probable human case of mad cow disea...,t case of probable mad cow disease listed post...
4,at least ## people were killed as typhoon sepa...,"orms lash chinese coast killing ## ; ###,### e..."


### Multinews

In [4]:
multinews_builder = tfds.builder("MultiNews")
multinews_builder.download_and_prepare()
multinews = multinews_builder.as_dataset()

In [5]:
multinews = tfds.as_dataframe(multinews["test"])
multinews = multinews.rename(columns={"document":"text", "summary":"summary"})
multinews['summary'] = multinews['summary'].str.decode("utf-8").str.replace("\xe2\x80\x93", "").str[2:].str.strip()
multinews['text'] = multinews['text'].str.decode("utf-8").str.strip()
multinews.head(5)

,text,summary
0,"Think back, everyone -- can you remember when ...",No matter how much you like Jason Bateman and ...
1,'Alien: Covenant': Film Review \n \n Michael ...,A spaceship arrives on a distant planet that l...
2,By REUTERS \n \n Photo: Reuters \n \n Charli...,France is on lockdown today after a satirical ...
3,These crawls are part of an effort to archive ...,Just when you thought the Republican president...
4,The estranged wife of acclaimed concert pianis...,"Sofya Tsygankova, the estranged wife of famed ..."


## Benchmark

In [12]:
results = {}

### Billsum

#### Classic BERTScore

In [6]:
bert_scores, bert_runtime = BERTScoreDynamicSampleTest(billsum_test)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaM

#### Static BERTScore

In [7]:
w2v = serialized_to_model(r'C:\Pro\Stages\A4 - DVRC\Work\Models\serialized_w2v.pkl')

In [8]:
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(billsum_test, w2v, 3, withIdf = False)

#### BARTScore

In [9]:
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(billsum_test, limit=3)

#### Appending

In [10]:
billsum_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime]}

In [13]:
results['billsum'] = billsum_results

### Multinews

In [14]:
bert_scores, bert_runtime = BERTScoreDynamicSampleTest(multinews)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(multinews, w2v, 3, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(multinews, limit=3)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaM

In [15]:
multinews_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime]}
results['multinews'] = multinews_results

### Gigaword

In [30]:
bert_scores, bert_runtime = BERTScoreDynamicSampleTest(gigaword)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(gigaword, w2v, 3, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(gigaword, limit=3)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaM

In [31]:
gigaword_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime]}
results['gigaword'] = gigaword_results

## Evalutation

### Runtime Table

In [33]:
billsum_runtimes = [results["billsum"]["BERTScore"][1], results["billsum"]["Custom"][1], results["billsum"]["Bart"][1]]
multinews_runtimes = [results["multinews"]["BERTScore"][1], results["multinews"]["Custom"][1], results["multinews"]["Bart"][1]]
gigaword_runtimes = [results["gigaword"]["BERTScore"][1], results["gigaword"]["Custom"][1], results["gigaword"]["Bart"][1]]

runtimeDf = pd.DataFrame({"Billsum": billsum_runtimes, 
                          "Multinews": multinews_runtimes,
                          "Gigaword": gigaword_runtimes}, 
                         index=["Roberta-24-layers", "Word2Vec", "BART-large-CNN"])
runtimeDf

,Billsum,Multinews,Gigaword
Roberta-24-layers,12.262710,9.243795,8.908763
Word2Vec,5.691295,13.544956,0.011086
BART-large-CNN,0.355920,0.398936,0.069543


### Quality evaluation

#### Billsum

In [17]:
billsumQualityDf = pd.concat((pd.DataFrame(results["billsum"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["billsum"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["billsum"]["Bart"][0], columns=["Bart"])),
                              axis=1)
billsumQualityDf

,W2V_P,W2V_R,W2V_F,Bert_P,Bert_R,Bert_F,Bart
0,0.646725,0.902619,0.753539,0.849191,0.781935,0.814176,0.053699
1,0.699015,0.941868,0.802470,0.818807,0.699549,0.754495,0.038453
2,0.652091,0.959232,0.776389,0.832273,0.697599,0.759008,0.038518


#### Multinews

In [18]:
multinewsQualityDf = pd.concat((pd.DataFrame(results["multinews"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["multinews"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["multinews"]["Bart"][0], columns=["Bart"])),
                              axis=1)
multinewsQualityDf

,W2V_P,W2V_R,W2V_F,Bert_P,Bert_R,Bert_F,Bart
0,0.648935,0.854094,0.737513,0.810646,0.794206,0.802342,0.028218
1,0.656565,0.880320,0.752154,0.841052,0.826047,0.833482,0.047064
2,0.651669,0.863386,0.742735,0.863537,0.827867,0.845326,0.055904


#### Gigaword

In [34]:
gigawordQualityDf = pd.concat((pd.DataFrame(results["gigaword"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["gigaword"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["gigaword"]["Bart"][0], columns=["Bart"])),
                              axis=1)
gigawordQualityDf

,W2V_P,W2V_R,W2V_F,Bert_P,Bert_R,Bert_F,Bart
0,0.530460,0.893533,0.665711,0.921882,0.861801,0.890830,0.013748
1,0.428361,0.616523,0.505500,0.864806,0.799132,0.830673,0.004452
2,0.426867,0.751144,0.544373,0.867719,0.825670,0.846172,0.001011
